In [ ]:
import numpy as np
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from numpy.random import choice
import copy
from matplotlib import cm
import matplotlib
import matplotlib.image as mpimg
import matplotlib.animation as animation
from IPython.display import HTML  # for embedded matplotlib animation
from math import *

matplotlib.rc('animation', html='html5')

In [ ]:
# GLobal variables

global GRID_SIZE, PHEROMONE_STRENGTH, PHEROMONE_FADE, ANT_VALUE, FOOD_SOURCE_VALUE, N_WORK, N_SEARCH

GRID_SIZE =10
PHEROMONE_STRENGTH = 1
PHEROMONE_FADE = 0.0001 
ANT_VALUE = 5 
FOOD_SOURCE_VALUE = 1000 
N_WORK = 5 
N_SEARCH =1



In [ ]:
class Grid:
    
    # para([Grid size, pheromone strength, pheromone fade, n_search, n_work])
    # de n_work en n_search worden later toegevoegd wegens errors
    def __init__(self, para):
        # Create board of specifc board size 
        self.grid = np.zeros((para[0], para[0]))
        self.grid_size = para[0]
        self.nest_value = 25
        self.food_source_value = 1000
        self.ant_value = 5
        self.pheromone_strength = para[1]
        self.pheromone_fade = para[2]
        self.n_work = para[3]
        self.n_search = para[4]
        self.ant_dict = {}
        self.nest_location = None
        self.food_location = {}
        self.total_cost = 0
        self.cost_per_step = 0.1
        self.return_reward = 1
        self.food_reward = 2*self.cost_per_step * (para[0]-1) +1
        

    # Get neighbour xy coordinates as tuple  
    def check_neighbours(self, coord, grid_size):
        # Initalize variables
        neighbours = []

        # List of relative position for all possible neighbours of a coordinate
        adj = [(-1, 1), (0, 1), (1, 1), (-1, 0), (1, 0), (-1, -1), (0,-1), (1,-1)]

        # Create list of all possible nieghbours
        for n in adj:
            neighbours.append(tuple(np.subtract(coord, n)))

        # In case of corners and edge pieces: find all neighbours that exist but are not valid for the specific 
        # board
        neigh = []
        for n_tup in neighbours:
            if n_tup[0] < 0 or n_tup[1] < 0:
                neigh.append(n_tup)
            elif n_tup[0] >= grid_size or n_tup[1] >= grid_size:
                neigh.append(n_tup)

        # Remove the found none valid neighbours 
        for i in neigh:
            neighbours.remove(i)

        # Return list of neighbours as tuple
        return neighbours

    # change value at specific cell, returns new grid
    def change_cell_value(self, coord, new_value):
        self.grid[coord[1]][coord[0]] = new_value
        return self.grid

    # return which value a cell has
    def get_kind(self, coord):
        x = coord[0]
        y = coord[1]
        return self.grid[y, x]
    
    # sets the new value of a cell
    def setKind(self, coord, value):
        x = coord[0]
        y = coord[1]
        if value is not 5 and value is not 25 and value < 100:
            return "Invalid value"
        self.grid[y, x] = value

    # if an ants arrives: sets value of ant in the cell 
    def antArrives(self, coord):
        x = coord[0]
        y = coord[1]
        self.grid[y, x] += self.ant_value

    # adds the pheromonevalue to the cell if ant leaves and removes the ant_value
    def antLeaves(self, coord):
        x = coord[0]
        y = coord[1]
        self.grid[y, x] -= self.ant_value
        self.addPheromone(coord)

    # adds phermone value to a cell
    def addPheromone(self, coord):
        x = coord[0]
        y = coord[1]
        self.grid[y, x] += pheromone_strength

    def pheromoneFade(self, coord):
        # The cell must already have pheromones
        x = coord[0]
        y = coord[1]
        
        # reaction-diffusion-type simulation (Moore neighbours)
        surroundPhero = 0
        for i in self.check_neighbours(coord, self.grid_size):
            if 0 <= self.get_kind(i) <= 1:
                surroundPhero += self.get_kind(i) 
        new_value = (self.get_kind((x, y)) * (1 - 8* self.pheromone_fade) + self.pheromone_fade*surroundPhero)
        self.grid[y, x] -= self.pheromone_strength 
        self.grid[y, x] += new_value

    # this is the function that calculates the fading of the pheromone
    def retrieveFood(self, coord):
        # The cell must be of the kind food source
        x = coord[0]
        y = coord[1]
        self.grid[y, x] -= self.food_value;
        
    # removes the value of food from the food stack value
    def setNestLocation(self, coord):
        x = coord[0]
        y = coord[1]
        self.nest_location = (x,y)
        self.setKind(coord, self.nest_value)

    # sets the coordinates of food source 
    def setFoodSource(self, coord):
        x = coord[0]
        y = coord[1]
        self.setKind(coord, self.food_source_value)
        self.food_location.append(coord)

    # returns the distance of an coordinate to the origin (nest/food source)
    def origin_distance(self, coord, origin):
        val = np.square(coord[0]-origin[0])+np.square(coord[1]-origin[1])
        return np.sqrt(val)
       
    # returns the list of possible steps for an ant
    def possible_steps_list(self, coord, origin):
        pn = self.check_neighbours(coord, self.grid_size)
        dist = self.origin_distance(coord, origin)
        
        best_neigh = []
        for n in pn:
            distance = self.origin_distance(n, origin)
            if distance > dist:
                best_neigh.append(n)
        
        return best_neigh
        
    # returns the step of a worker ant
    def decide_step_worker(self, coord, origin):
        possible_steps = self.possible_steps_list(coord, origin)
        pos_step = []
        pos_step_sum = 0
        
        for p in possible_steps:
            if 0 < self.get_kind(p) <= 1:
                pos_step.append(p)
                pos_step_sum += self.get_kind(p)
                
        probability_distribution = []
        
        for pos in pos_step:
            self.get_kind(pos)
            probability_distribution.append(self.get_kind(pos)/pos_step_sum)
        
        if pos_step == []:
            pos_step = possible_steps
            prob_value = 1/len(possible_steps)
            probability_distribution = [prob_value]*len(possible_steps)
        
        [index] = choice(range(len(pos_step)), 1, p=probability_distribution)

        return pos_step[index]
    
    # returns the step of a searcher ant
    def decide_step_search(self, coord, origin):
        possible_steps = self.check_neighbours(coord, self.grid_size)
        
        pos_step = []
        for p in possible_steps:
            if self.get_kind(p) in self.food_location.keys():
                
                return self.get_kind(p)
            if self.get_kind(p) == 0 or self.ant_value < self.get_kind(p) < self.ant_value+self.pheromone_strength:
                pos_step.append(p)
        
        if pos_step == []:
            pos_step = possible_steps

        return random.choice(pos_step)
    
    # makes the ant set a step and calculates new total cost
    def antSetStep(self, coord, nextCoord):
        x = coord[0]
        y = coord[1]
        x_next = nextCoord[0]
        y_next = nextCoord[1]
        self.grid[y_next, x_next] += self.ant_value
        self.grid[y, x] -= self.ant_value
        self.grid[y, x] += self.pheromone_strength
        self.total_cost += self.cost_per_step

    # decides the route back of the search ants, once they have located the food source
    def routeBack(self, coord, origin):
        x = coord[0]
        y = coord[1]

        x_ant = x
        y_ant = y

        x_orig = origin[0]
        y_orig = origin[1]

        dx = x - x_orig
        dy = y - y_orig

        # Three possibilities: ant is on the same x or y coordinate, in which case it
        # can walk back in a straight line.
        # Or the x-distance is the same as the y-distance, in whcih case the ant can 
        # walk back in diagonal line.
        # Else, the ant should walk back in a partly diagonal, partly straight line.

        route_back_list = []
        
        while x != self.nest_location[0] or y != self.nest_location[1]:
            if x == x_orig:
                y_ant = y - 1
                route_back_list.append((x, y_ant))
                y = y_ant
                continue
                
            if y == y_orig:
                x_ant = x - 1
                route_back_list.append((x_ant, y))
                x = x_ant
                continue

            if np.abs(dx) == np.abs(dy):
                x_ant = x - 1
                y_ant = y - 1
                route_back_list.append((x_ant, y_ant))
                x = x_ant
                y = y_ant
                continue
                
            else:
                x_ant = x - 1
                y_ant = y - 1
                route_back_list.append((x_ant, y_ant))
                x = x_ant
                y = y_ant

        return route_back_list
            
    #adds worker ant        
    def add_work_ant(self):
        self.ant_dict[self.nest_location] = self.nest_location, 'w'
    
    #adds all searcher ants
    def release_searches(self):
        while n_search != 0:
            n_search -= 1
            self.add_search_ant()
    
    # adds one search ant
    def add_search_ant(self):
        self.ant_dict[self.nest_location] = self.nest_location, 's'
        
    # gives the origin of an ant
    def get_ant_origin(self, coord):
        return self.ant_dict[coord]
        
    # add amount of food to the food stack
    def add_food_location(self, coord, amount):
        self.food_location[coord] = amount
        
    # steps to do when a search ant returns
    def return_of_search(self):
        # the ant is on the nest location
        if self.nest_location in self.ant_dict.keys():
            
            # get the origin of the and and type of ant
            origin = self.ant_dict[self.nest_location][0]
            ant_type = self.ant_dict[self.nest_location][1]
            
            # checks is the origin is the food location and and is searcher
            if origin in self.food_location.keys() and ant_type == 's':
                
                # adds worker ants until all ants are on their way to food
                while self.n_work != 0:
                    self.add_work_ant()
                    self.n_work -= 1


    # update board
    def renew_board(self):
        current_board = copy.deepcopy(self)
        
        for i in range(len(self.grid)):
            for j in range(len(self.grid)):
                curr_cell = (j,i)
                cell_value = self.get_kind((j,i))
                
                # If cell is empty
                if cell_value == 0:
                    continue
                
                # If cell contains an ant
                if curr_cell in current_board.ant_dict.keys():
                    origin = current_board.ant_dict[curr_cell][0]
                    type_ant = current_board.ant_dict[curr_cell][1]
                    
                    # If cell contains a foodsource, change origin when there is also an ant and lower the 
                    # amount of food left and lower cost
                    if curr_cell in current_board.food_location.keys():
                        self.ant_dict[curr_cell] = curr_cell, type_ant
                        self.total_cost -= self.food_reward
                        self.food_location[curr_cell] =  self.food_location[curr_cell] - 1
                        
                    # If cell contains a nest, change origin when there is also an ant
                    # If ant is search ant and back in nest than delete
                    if curr_cell == current_board.nest_location:
                        self.ant_dict[curr_cell] = curr_cell, type_ant 
                        if type_ant == 's' and origin != current_board.nest_location:
                            del self.ant_dict[curr_cell] 
                            self.total_cost -= self.return_reward
                    
                    # Check if worker ant
                    if type_ant == 'w':
                        new_cell = current_board.decide_step_worker(curr_cell, origin)
                        self.antSetStep(curr_cell, new_cell)
                        del self.ant_dict[curr_cell]
                        self.ant_dict[new_cell] = origin, type_ant
                        
                    # Check if search ant
                    if type_ant == 's':
                        if origin == current_board.nest_location:
                            new_cell = current_board.decide_step_search(curr_cell, origin)
                            self.antSetStep(curr_cell, new_cell)
                            del self.ant_dict[curr_cell]
                            self.ant_dict[new_cell] = origin, type_ant
                        if origin != current_board.nest_location:
                            new_cell = current_board.routeBack(curr_cell, origin)[0]
                            self.antSetStep(curr_cell, new_cell)
                        
                # If cell is empty but contains a pheromone
                if 0 < cell_value <= 1:
                    self.pheromoneFade(curr_cell)

                
    def showGrid(self):
        # Amount of different colors for pheromones
        pheromone_amount = 80
        
        # Blue gradient for pheromones: the darker the blue, the more pheromones
        pher_colors = cm.Blues(np.linspace(0, 1, num=pheromone_amount)).tolist()
        pher_bounds = np.linspace(0,1,num=pheromone_amount+1).tolist()
        
        # Other color: nothing, ant, nest and food source
        other_colors = ['white', 'black', 'white', 'peru', 'white', 'forestgreen']
        
        # Boundaries for the values of the other colors
        other_bounds = [1.00001, self.ant_value, 6.00001, 9.99999, self.nest_value + 0.01, 99.99999, 100.0]
        total_colors = sum([pher_colors, other_colors], [])
        total_bounds = sum([pher_bounds, other_bounds], [])
        
        # If the value is 0, the cell is white
        total_colors[0] = 'white'
        
        # Create a cmap of all the colors
        cmap = mpl.colors.ListedColormap(total_colors)
        norm = mpl.colors.BoundaryNorm(total_bounds, cmap.N)
        
        img = plt.imshow(self.grid,interpolation='nearest', cmap=cmap, norm=norm)
        plt.xticks([], [])
        plt.yticks([], [])
        plt.show()   

world = Grid([10, 1, 0.0000001, 5, 1])

world.setNestLocation((0,0))
world.add_food_location((2,4), 4)
world.add_search_ant()
print(world.return_of_search())

world.showGrid()

# for i in range(20):
#     world.renew_board()
#     world.showGrid()

In [ ]:
class Simulation:
    
    def __init__(self, para):
        self.grid = para[0]
        self.ant_value = para[1]
        self.nest_value = para[2]
        self.food_source_value = para[3]
        
    
    def showGrid(self):
        # Amount of different colors for pheromones
        pheromone_amount = 80
        
        # Blue gradient for pheromones: the darker the blue, the more pheromones
        pher_colors = cm.Blues(np.linspace(0, 1, num=pheromone_amount)).tolist()
        pher_bounds = np.linspace(0,1,num=pheromone_amount+1).tolist()
        
        # Other color: nothing, ant, nest and food source
        other_colors = ['white', 'black', 'white', 'peru', 'white', 'forestgreen']
        
        # Boundaries for the values of the other colors
        other_bounds = [1.00001, self.ant_value, 6.00001, 9.99999, self.nest_value + 0.01, 99.99999, 100.0]
        total_colors = sum([pher_colors, other_colors], [])
        total_bounds = sum([pher_bounds, other_bounds], [])
        
        # If the value is 0, the cell is white
        total_colors[0] = 'white'
        
        # Create a cmap of all the colors
        cmap = mpl.colors.ListedColormap(total_colors)
        norm = mpl.colors.BoundaryNorm(total_bounds, cmap.N)
        
        img = plt.imshow(self.grid,interpolation='nearest', cmap=cmap, norm=norm)
        img.set_array(img)
        return img,

    
    # This is the animation function for the grids
    def animation(self):
        fig = plt.figure()
        plt.axis('off')
        ants = mpatches.Patch(color='black', label='Ants')
        nest = mpatches.Patch(color='peru', label='Nest')
        food = mpatches.Patch(color='forestgreen', label='Food source')
        phero = mpatches.Patch(color='blue', label='Gradient of pheromone')
        plt.legend(handles=[ants, nest, food, phero])
        
        animat = animation.FuncAnimation(fig, self.showGrid(), 
                                   save_count = self.grid.shape[2], 
                                   interval=50, blit=False)
        
        plt.show()

In [ ]:
# main function

# eerst initialize board with search ants
# dan zodra  1 search ant terug is : worker ants
# dit herhalen totdat de foodstack leeg is!
# in elke stap moet de visualisatie plaatsvinden

#  while self.food_source_value != 0:
#             for i in self.n_search:
#                 init_step = decide_step_search(self.nest_location, self.nest_location)
#                 ant_set_step(self.nest_location, init_step)
#                 renew_board()
#                 self.add_search_ant()
#                 self.renew_board()
#                 self.showGrid()
#             # if food location is nog niet gevonde
            
#             # if food location is gevonden & search ants zijn terug
            
#             if
#             self.renew_board()
#             self.showGrid(
   

    
def main():
    while FOOD_SOURCE_VALUE != 0:
        for i in N_SEARCH:
            N_SEARCH -= 1
            Grid([GRID_SIZE, PHEROMONE_STRENGTH, PHEROMONE_FADE])
            Simulation([])
        Grid([GRID_SIZE, PHEROMONE_STRENGTH, PHEROMONE_FADE]) 
        
    
    
Writer = animation.writers['ffmpeg']
writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
        
grid_ani.save('AntsColor.mp4', writer=writer)

HTML(grid_ani.to_html5_video())
